# MistralAI

Weave automatically tracks and logs LLM calls made via the [MistralAI Python library](https://github.com/mistralai/client-python). 

> **Note: We support the new Mistral v1.0 SDK, check the migration guide [here](https://github.com/mistralai/client-python/blob/main/MIGRATION.md)**



## Traces

It’s important to store traces of LLM applications in a central database, both during development and in production. You’ll use these traces for debugging, and as a dataset that will help you improve your application.

Weave will automatically capture traces for [mistralai](https://github.com/mistralai/client-python). You can use the library as usual, start by calling `weave.init()`:


In [ ]:
!pip install weave mistralai -qqq

In [ ]:
from mistralai import Mistral
import weave
import os
import getpass

print("Enter your Mistral API key:")
os.environ["MISTRAL_API_KEY"] = getpass.getpass()


If not already logged in, you will be prompted to log into wandb and authorize using your wandb account in the next step.

In [ ]:
# Initialize Weave
weave.init("quickstart_mistral")

In [ ]:
model = "mistral-large-latest"

client = Mistral(api_key=os.environ["MISTRAL_API_KEY"])

messages = [
    {
        "role": "user",
        "content": "What is the best French cheese?",
    },
]

chat_response = client.chat.complete(
    model=model,
    messages=messages,
)



Weave will now track and log all LLM calls made through the MistralAI library. You can view the traces in the Weave web interface.

[![mistral_trace.png](https://weave-docs.wandb.ai/assets/images/mistral_trace-3fa22a88515d57264ee6099e52aa206a.png)](https://wandb.ai/capecape/mistralai_project/weave/calls)



## Wrapping with your own ops

Weave ops make results *reproducible* by automatically versioning code as you experiment, and they capture their inputs and outputs. Simply create a function decorated with [`@weave.op`](https://weave-docs.wandb.ai/guides/tracking/ops) that calls into [`mistralai.client.MistralClient.chat()`](https://docs.mistral.ai/capabilities/completion/) and Weave will track the inputs and outputs for you. Let's see how we can do this for our cheese recommender:


In [ ]:
@weave.op
def cheese_recommender(region:str, model:str) -> str:
    "Recommend the best cheese in a given region"
    
    messages = [
        {
            "role": "user",
            "content": f"What is the best cheese in {region}?",
        },
    ]

    chat_response = client.chat.complete(
        model=model,
        messages=messages,
    )
    return chat_response.choices[0].message.content

cheese_recommender(region="France", model=model)
cheese_recommender(region="Spain", model=model)
cheese_recommender(region="Netherlands", model=model)




[![mistral_ops.png](https://weave-docs.wandb.ai/assets/images/mistral_ops-8e251fbc90ce13b865c2140b60445abf.png)](https://wandb.ai/capecape/mistralai_project/weave/calls)


## Create a `Model` for easier experimentation

Organizing experimentation is difficult when there are many moving pieces. By using the [`Model`](https://weave-docs.wandb.ai/guides/core-types/models) class, you can capture and organize the experimental details of your app like your system prompt or the model you're using. This helps organize and compare different iterations of your app. 

In addition to versioning code and capturing inputs/outputs, [`Model`](https://weave-docs.wandb.ai/guides/core-types/models)s capture structured parameters that control your application’s behavior, making it easy to find what parameters worked best. You can also use Weave Models with [`serve`](https://weave-docs.wandb.ai/guides/tools/serve), and [`Evaluation`](https://weave-docs.wandb.ai/guides/core-types/evaluations)s.

In the example below, you can experiment with `model` and `country`. Every time you change one of these, you'll get a new _version_ of `CheeseRecommender`. 



In [ ]:
class CheeseRecommender(weave.Model): # Change to `weave.Model`
    model: str
    temperature: float

    @weave.op
    def predict(self, region:str) -> str: # Change to `predict`
        "Recommend the best cheese in a given region"
        
        client = Mistral(api_key=os.environ["MISTRAL_API_KEY"])

        messages = [
            {
                "role": "user",
                "content": f"What is the best cheese in {region}?",
            },
        ]

        chat_response = client.chat.complete(
            model=model,
            messages=messages,
            temperature=self.temperature
        )
        return chat_response.choices[0].message.content

cheese_model = CheeseRecommender(
    model=model,
    temperature=0.0
    )
result = cheese_model.predict(region="France")
print(result)


[![mistral_model.png](https://weave-docs.wandb.ai/assets/images/mistral_model-0a89ef6374318020e9da24e9d74f0d52.png)](https://wandb.ai/capecape/mistralai_project/weave/models)
